### Before starting a new run, create a new qual to give to all workers

(Note that this is different from the *participation* qual, which is assigned to everyone across all runs)

(Also, in case you need it, the link to manage the quals via the Web UI: https://requester.mturk.com/qualification_types )

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import dotenv

from mtclient import MTClient
import mtglobals

In [3]:
client = MTClient()

Loading environment variables from .env
Using LIVE MTurk requester API
Your account balance is 4775.12


### (a) Get info for the previous (most recent) custom qual

In [4]:
qual_info = mtglobals.get_current_qual()

### (c) Create the new qual via MTurk API

In [5]:
dotenv.load_dotenv(mtglobals.dotenv_fpath, override=True)
qual_prefix = os.getenv("WAVE_QUAL_PREFIX")
if qual_info is None:
    # No info on previous quals, so create first qual
    new_qual_name = f"{qual_prefix}_00"
    # And create it using API
    result = client.create_new_qual(new_qual_name)
else:
    # Increment the suffix and create
    last_qual_name = qual_info['qual_name']
    last_qual_num = qual_info['last_qual_num']
    last_offer_amt = qual_info['last_offer_amt']
    last_qual_suffix = int(last_qual_name.split("_")[-1])
    new_qual_suffix = str(last_qual_suffix + 1).zfill(2)
    new_qual_name = f"{qual_prefix}_{new_qual_suffix}"
    result = client.create_new_qual(new_qual_name)
print(f"Created first qualification {result['qual_name']} with ID {result['qual_id']}")

Created first qualification Workplace_Survey_18 with ID 30M5Q3A44NCL1C9BMLI9I5P0L1PK7M


In [6]:
# This is left over from an old version -- we don't technically need to keep track of
# the previous wage anymore, but I'm keeping it in just for ease of debugging since
# this makes it so the log files will always record each generated wage. Here it's
# just chosen at random, it'll be re-sampled anyways when the first stage-2 HIT is launched
start_offer_amt = mtglobals.draw_random_wage()

### (d) And update the locally-stored info on the current qual

In [7]:
mtglobals.update_current_qual(result['qual_name'], result['qual_id'], 0, start_offer_amt)

Current qual info in ..\results_2stage\current_qual.txt updated to be: Workplace_Survey_18,30M5Q3A44NCL1C9BMLI9I5P0L1PK7M,0,0.98


### Now run 01b_PostStage1.ipynb

### (old version)

(potentially useful if API acts weird and you need to just get the id for the most-recently-created qual)

In [12]:
# Get the suffix
qual_suffix = int(qual_info['qual_name'][-2:])
int(qual_suffix)
# Increment
new_suffix = qual_suffix + 1
new_suffstr = str(new_suffix).zfill(2)
# Create the new qual name
new_qual_name = "Workplace_Survey_Custom" + new_suffstr
new_qual_name
print(new_qual_name)
# Create the actual qual via mturk api
if not mtglobals.qual_exists(client, new_qual_name):
    result_name, result_id = mtglobals.create_new_qual(client, new_qual_name)
    result_id = mtglobals.get_qual_id(client, new_qual_name)
    # And set as the new current qual
    mtglobals.update_current_qual(new_qual_name, result_id, 0, '0.50')
else:
    # If it already exists, just get the id and update the current qual
    print("Qual already exists, updating current qual.txt")
    result_id = mtglobals.get_qual_id(client, new_qual_name)
    mtglobals.update_current_qual(new_qual_name, result_id, 0, '0.50')

Workplace_Survey_Custom14
Qual already exists, updating current qual.txt
Current qual updated to be: Workplace_Survey_Custom14,376OKSJ89SYRLC8IL4XY8ZX2HAYX7I,0,0.50


### Sigh. No idea why it works sometimes and doesn't work other times, but just run this to update the currentqual.txt

In [11]:
# If it already exists, just get the id and update the current qual
print("Qual already exists, updating current qual.txt")
result_id = mtglobals.get_qual_id(client, new_qual_name)
mtglobals.update_current_qual(new_qual_name, result_id, 0, '0.50')

Qual already exists, updating current qual.txt


Exception: No matching quals found!